<a href="https://colab.research.google.com/github/eyv7jz/ds1002-eyv7jz/blob/main/API_Cals_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd

In [4]:
def get_API_response(url, response_type):
  try:
      response = requests.get(url)
      response.raise_for_status()

  except requests.exceptions.HTTPError as errh:
      return "An Http Error occurred: " + repr(errh)
  except requests.exceptions.ConnectionError as errc:
      return "An Error Connecting to the API occurred: " + repr(errc)
  except requests.exceptions.Timeout as errt:
      return "A Timeout Error occurred: " + repr(errt)
  except requests.exceptions.RequestException as err:
      return "An Unknown Error occurred: " + repr(err)

  if response_type == 'json':
      result = json.dumps(response.json(), sort_keys=True, indent=4)
  elif response_type == 'dataframe':
      result = pd.json_normalize(response.json())
  else:
      result = "An unhandled error has occurred!"

  return result

In [5]:
name = input("Enter a country name: ")
URL = f"https://restcountries.com/v3.1/name/{name}"
response_type = ['json', 'dataframe']
df = get_API_response(URL, response_type[1])

if isinstance(df, str):
  print("Enter a real country name. ")
else:
  df_final = df[['name.common', 'capital', 'population']]
  while True:
    additional_countries = input("Do you want to enter additional countries? ")
    if additional_countries == 'yes':
      name = input("Enter a country name: ")
      URL = f"https://restcountries.com/v3.1/name/{name}"
      df = get_API_response(URL, response_type[1])
      if isinstance(df, str):
        print("Enter the name of a real country.")
      else:
        df_new_entry = df[['name.common', 'capital', 'population']]
        df_final = pd.concat([df_final, df_new_entry], ignore_index = True)
    elif additional_countries == 'no':
      df_final.rename(columns={'name.common': 'Common Name', 'capital': 'Capital City', 'population': 'Population'}, inplace = True)
      print(df_final)
      break

Enter a country name: Germany
Do you want to enter additional countries? yes
Enter a country name: Austria
Do you want to enter additional countries? yes
Enter a country name: Chile
Do you want to enter additional countries? yes
Enter a country name: Portugal
Do you want to enter additional countries? no
  Common Name Capital City  Population
0     Germany     [Berlin]    83240525
1     Austria     [Vienna]     8917205
2       Chile   [Santiago]    19116209
3    Portugal     [Lisbon]    10305564


In [6]:
df_final.to_json('Country_Data.json', index = True)